<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

The sample was tested with TFX version: 0.21.4 and KFP version: 0.4.0. Make sure that these versions are installed in your environment.

In [ ]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

Add a path to skaffold.

In [ ]:
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

## Building and deploying the pipeline

You will use TFX CLI to compile and deploy the pipeline. As explained in the previous section, the environment specific settings can be provided through a set of environment variables and embedded into the pipeline package at compile time.

### Configure settings


In [ ]:
PROJECT_ID = 'mlops-dev-env'
GCP_REGION = 'us-central1'
ENDPOINT = '309f723963874a47-dot-us-central2.pipelines.googleusercontent.com'
ARTIFACT_STORE_URI = 'gs://mlops-dev-workspace'
PIPELINE_NAME = 'custom_component'
DATA_ROOT_URI = 'gs://mlops-dev-workspace/data/taxi'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)

In [ ]:
!gsutil cat {DATA_ROOT_URI}/data.csv

### Compile the pipeline


In [ ]:
%env PROJECT_ID={PROJECT_ID}
%env KUBEFLOW_TFX_IMAGE={CUSTOM_TFX_IMAGE}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env GCP_REGION={GCP_REGION}
%env PIPELINE_NAME={PIPELINE_NAME}

In [ ]:
!tfx pipeline compile --engine kubeflow --pipeline_path dataflow_runner.py

### Deploy the pipeline package to AI Platform Pipelines

After the pipeline code compiles without any errors you can use the `tfx pipeline create` command to perform the full build and deploy the pipeline. 


If you need to redeploy the pipeline you can use first delete the previous version using `tfx pipeline delete` or you can update the pipeline in-place using `tfx pipeline update`.

To delete the pipeline:

`tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}`

To update the pipeline:

`tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}`

### Create and monitor a pipeline run
After the pipeline has been deployed, you can trigger and monitor pipeline runs using TFX CLI or KFP UI.

To submit the pipeline run using TFX CLI:

In [ ]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

To list all active runs of the pipeline:

In [ ]:
!tfx run list --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

To retrieve the status of a given run:

In [ ]:
RUN_ID='[YOUR RUN ID]'

!tfx run status --pipeline_name {PIPELINE_NAME} --run_id {RUN_ID} --endpoint {ENDPOINT}